In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
df = pd.read_csv('./data/highway_1812.csv')
df.head()

,집계일자,집계시간,영업소명,입구출구구분,통행량
0,20181201,0,가락,0,143
1,20181201,0,가락,1,178
2,20181201,0,가락2,0,38
3,20181201,0,가락2,1,41
4,20181201,0,가산,0,58


In [7]:
df.tail()

,집계일자,집계시간,영업소명,입구출구구분,통행량
585059,20181231,23,황전,1,8
585060,20181231,23,회인,0,40
585061,20181231,23,회인,1,61
585062,20181231,23,횡성,0,37
585063,20181231,23,횡성,1,79


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585064 entries, 0 to 585063
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   집계일자    585064 non-null  int64 
 1   집계시간    585064 non-null  int64 
 2   영업소명    585064 non-null  object
 3   입구출구구분  585064 non-null  int64 
 4   통행량     585064 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 22.3+ MB


### pd.to_datetime(df['col'])
    *str상태에서 변경해야함
#### dt.dayofweek
    *dt.col.dt.dayofweek -> 월요일 = 0 시작

In [11]:
df['집계일자'] = df.집계일자.astype('str')
df['집계일자'] = pd.to_datetime(df['집계일자'])

df

,집계일자,집계시간,영업소명,입구출구구분,통행량
0,2018-12-01,0,가락,0,143
1,2018-12-01,0,가락,1,178
2,2018-12-01,0,가락2,0,38
3,2018-12-01,0,가락2,1,41
4,2018-12-01,0,가산,0,58
...,...,...,...,...,...
585059,2018-12-31,23,황전,1,8
585060,2018-12-31,23,회인,0,40
585061,2018-12-31,23,회인,1,61
585062,2018-12-31,23,횡성,0,37


In [12]:
df.집계일자.dt.dayofweek         # dt.dayofweek

0         5
1         5
2         5
3         5
4         5
         ..
585059    0
585060    0
585061    0
585062    0
585063    0
Name: 집계일자, Length: 585064, dtype: int64

##### 월요일의 데이터만 출력

In [59]:
df[df.집계일자.dt.dayofweek == 0]            

,집계일자,집계시간,영업소명,입구출구구분,통행량
37967,2018-12-03,0,가락,0,115
37968,2018-12-03,0,가락,1,130
37969,2018-12-03,0,가락2,0,21
37970,2018-12-03,0,가락2,1,15
37971,2018-12-03,0,가산,0,49
...,...,...,...,...,...
585059,2018-12-31,23,황전,1,8
585060,2018-12-31,23,회인,0,40
585061,2018-12-31,23,회인,1,61
585062,2018-12-31,23,횡성,0,37


##### '서울(특)','동서울'톨게이트의 데이터 추출

In [17]:
df[df.영업소명.isin(['서울(특)','동서울'])]      #.isin()  : 원하는 값만 추출할때 쓰임

,집계일자,집계시간,영업소명,입구출구구분,통행량
250,2018-12-01,0,동서울,0,23
251,2018-12-01,0,동서울,1,62
449,2018-12-01,0,서울(특),0,54
450,2018-12-01,0,서울(특),1,72
1037,2018-12-01,1,동서울,0,14
...,...,...,...,...,...
584068,2018-12-31,22,서울(특),1,122
584579,2018-12-31,23,동서울,0,50
584580,2018-12-31,23,동서울,1,155
584761,2018-12-31,23,서울(특),0,52


##### 영업소 : '통영 ' or '북통영' / 요일은 토요일

In [24]:
df[(df.영업소명.isin(['통영','북통영'])) & (df.집계일자.dt.dayofweek == 6)]

,집계일자,집계시간,영업소명,입구출구구분,통행량
19371,2018-12-02,0,북통영,0,26
19372,2018-12-02,0,북통영,1,37
19695,2018-12-02,0,통영,0,167
19696,2018-12-02,0,통영,1,245
20157,2018-12-02,1,북통영,0,12
...,...,...,...,...,...
567556,2018-12-30,22,통영,1,384
568014,2018-12-30,23,북통영,0,25
568015,2018-12-30,23,북통영,1,35
568337,2018-12-30,23,통영,0,147


##### 요일별 통행량을 계산하고, 통행량이 가장 많은 요일 은 언제인가?

In [58]:
df.groupby(df.집계일자.dt.dayofweek).통행량.sum().sort_values(ascending=False)    # 5 : 토요일

집계일자
5    40720396
0    37598555
6    33777671
4    33555256
2    31383054
3    30972330
1    29181688
Name: 통행량, dtype: int64

##### 7시에서 9시사이 입구 통행량이 가장 많은 영업소는?
    *입구출구구분 : 0 = 입구통행량

In [57]:
df[(df.집계시간.isin([7,8,9])) & (df.입구출구구분 == 0)].groupby('영업소명').통행량.sum().sort_values().tail(1)

영업소명
대천    512967
Name: 통행량, dtype: int64